In [34]:
from requests import get
from lxml import etree
import io
import pandas as pd

url_serieA = "https://www.betclic.fr/football/italie-serie-a-e6"
url_liga = "https://www.betclic.fr/football/espagne-liga-primera-e7"
url_premierLeague = "https://www.betclic.fr/football/angl-premier-league-e3"
url_ligue1 = "https://www.betclic.fr/football/ligue-1-conforama-e4"
url_bundesliga = "https://www.betclic.fr/football/allemagne-bundesliga-e5"



headers = {"encoding": "utf-8"} 
page_serieA = get(url_serieA, headers=headers).content
page_liga = get(url_liga, headers=headers).content
page_premierLeague = get(url_premierLeague, headers=headers).content
page_ligue1 = get(url_ligue1, headers=headers).content
page_bundesliga = get(url_bundesliga, headers=headers).content

In [2]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [39]:
def clean_body(raw_page):

    start = '<div class="entry day-entry grid-9 nm"'
    end = 'var refreshTime = 20000;'

    page = raw_page.decode()

    body = page[page.find(start):page.rfind(end)]
    body = body.replace("&", "")
    body = body.replace("<a", "")
    body = body.replace("</a>", "")

    body = rreplace(body, "</div>", "", 2)
    body = rreplace(body, '<script type="text/javascript" language="javascript">', "", 1)
    body = rreplace(body, '</section>', "", 1)

    body = "<root>"+body+"</root>"
    return body

In [40]:
def crawl_info(cleaned_body, league):

    byte = str.encode(cleaned_body)

    file = io.BytesIO(byte)

    data = pd.DataFrame(columns=["match","date","hour","H","D","A","league"])

    match=""
    date=""
    hour=""
    odds = [0,0,0]
    i = 0

    for event, element in etree.iterparse(file):
        if element.tag == "time":
            date = element.get("datetime")
        if element.tag == "div" and element.get("class") == "hour":
            hour = element.text
        if element.tag == "div" and element.get("class") == "match-name":
            match = element.text.splitlines()[2].lstrip()

        if element.tag == "div" and element.get("class") == "match-odds":
            children = element.getchildren() 
            c=0

            for child in children:
                span = child.getchildren()[0]
                odds[c] = span.text
                c+=1
            data.loc[i] = [match, date, hour, odds[0], odds[1], odds[2], league]
            i+=1


    return data



In [32]:
def get_cotes(page, league):
    return crawl_info(clean_body(page), league)

In [54]:
cotes = get_cotes(page_ligue1,"ligue1").append(get_cotes(page_bundesliga,"bundesliga"), ignore_index=True)
cotes = cotes.append(get_cotes(page_liga,"liga"), ignore_index=True)
cotes = cotes.append(get_cotes(page_serieA,"serieA"), ignore_index=True)
cotes = cotes.append(get_cotes(page_premierLeague,"premierLeague"), ignore_index=True)
cotes = cotes.sort_values(by='date')
print(cotes.head(20))

                                match       date   hour     H     D     A  \
0                      Nîmes - Rennes  2020-1-15  19:00  3,25  3,15  2,35   
1                      Amiens - Reims  2020-1-15  19:00  2,82  2,88  2,85   
2                   Monaco - Paris SG  2020-1-15  21:00  6,00  5,15  1,45   
13   Schalke 04 - Borussia M'gladbach  2020-1-17  20:30  2,42  3,40  2,62   
40                   Leganes - Getafe  2020-1-17  21:00  2,80  2,75  2,88   
87                Newcastle - Chelsea  2020-1-18  18:30  5,80  4,35  1,50   
42              Real Madrid - Séville  2020-1-18  16:00  1,58  4,15  5,00   
43          Osasuna - Real Valladolid  2020-1-18  18:30  1,87  3,25  4,35   
44            Eibar - Atletico Madrid  2020-1-18  21:00  4,85  3,25  1,80   
60                  Lazio - Sampdoria  2020-1-18  15:00  1,40  4,60  7,20   
61                  Sassuolo - Torino  2020-1-18  18:00  2,70  3,35  2,50   
62                Naples - Fiorentina  2020-1-18  20:45  1,62  3,95  5,00   